In [1]:
# Data processing
import pandas as pd

# Set pandas to show all columns in the dataframes
pd.set_option('display.max_columns', None)

In [2]:
cols = ["Name", "Start", "Size"]
data = pd.read_csv("SAVEDATA1000.csv", dtype=str, usecols=cols)
data

,Name,Start,Size
0,struct mhw_ib_save mhw_save,0h,AC30E0h
1,struct mhw_save_header header,0h,40h
2,u32 magic,0h,4h
3,u32 unknown0,4h,4h
4,u32 unknown1,8h,4h
...,...,...,...
5359452,u64 zero[215540],AC30B8h,8h
5359453,u64 zero[215541],AC30C0h,8h
5359454,u64 zero[215542],AC30C8h,8h
5359455,u64 zero[215543],AC30D0h,8h


In [3]:
def intdef(x : str, default = -1):
    try:
        return int(str(x)[0:-1], 16)
    except:
        return default


data["Start"] = data["Start"].apply(intdef)
data["Size"] = data["Size"].apply(intdef)
data

,Name,Start,Size
0,struct mhw_ib_save mhw_save,0,11284704
1,struct mhw_save_header header,0,64
2,u32 magic,0,4
3,u32 unknown0,4,4
4,u32 unknown1,8,4
...,...,...,...
5359452,u64 zero[215540],11284664,8
5359453,u64 zero[215541],11284672,8
5359454,u64 zero[215542],11284680,8
5359455,u64 zero[215543],11284688,8


In [4]:
# data["StartNext"] = data["Start"].shift(-1)
data["Remove"] = data["Start"] + data["Size"] > data["Start"].shift(-1)
data.drop(data[data["Remove"]].index, inplace=True)
data.drop(["Remove"], axis=1, inplace=True)

In [5]:
data["Unknown"] = data["Name"].str.contains(r"unknown|data\[")
unknown = data[data["Unknown"]]
unknown

,Name,Start,Size,Unknown
3,u32 unknown0,4,4,True
4,u32 unknown1,8,4,True
54,u32 unknown0,100,4,True
57,u8 unknown0[0],112,1,True
58,u8 unknown0[1],113,1,True
...,...,...,...,...
5143393,u8 unknown12[12324],9559827,1,True
5143394,u8 unknown12[12325],9559828,1,True
5143395,u8 unknown12[12326],9559829,1,True
5143396,u8 unknown12[12327],9559830,1,True


In [6]:
data["Zero"] = data["Name"].str.contains("zero")
zero = data[data["Zero"]]
zero

,Name,Start,Size,Unknown,Zero
34140,u16 zero0[0],25262,2,False,True
34141,u16 zero0[1],25264,2,False,True
34142,u16 zero0[2],25266,2,False,True
34143,u16 zero0[3],25268,2,False,True
34144,u16 zero0[4],25270,2,False,True
...,...,...,...,...,...
5359452,u64 zero[215540],11284664,8,False,True
5359453,u64 zero[215541],11284672,8,False,True
5359454,u64 zero[215542],11284680,8,False,True
5359455,u64 zero[215543],11284688,8,False,True


In [17]:
unknown_size = unknown['Size'].sum()
zero_size = zero['Size'].sum()
data_size = data['Size'].sum()
nonzero_mapped_size = data_size - zero_size - unknown_size
print(f"Unknown: {unknown_size}/{data_size} = {unknown_size/data_size:.8%}")
print(f"Zero: {zero_size}/{data_size} = {zero_size/data_size:.8%}")
print(f"Nonzero Mapped: {nonzero_mapped_size}/{data_size} = {nonzero_mapped_size/data_size:.8%}")

Unknown: 2603465/11284704 = 23.07074248%
Zero: 4845650/11284704 = 42.93998318%
Nonzero Mapped: 3835589/11284704 = 33.98927433%
